In [1]:
from elasticsearch import Elasticsearch

In [2]:
config = {'host':'localhost', 'port':9200}
es = Elasticsearch([config])

In [3]:
import json
wiki_path = "/opt/ml/mrc-level2-nlp-11/data/wikipedia_documents.json"
with open(wiki_path, "r", encoding="utf-8") as f:
    wiki_json = json.load(f)
    print(wiki_json["0"])


{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}


In [4]:
wiki_contexts = [{"id": val["document_id"], "title": val["title"], "context" : val["text"]} for key, val in wiki_json.items()]
wiki_contexts[:1]

[{'id': 0,
  'title': '나라 목록',
  'context': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'}]

In [5]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.2

In [6]:
import time
from subprocess import Popen, PIPE, STDOUT
import os
path_to_elastic = "elasticsearch-7.9.2/bin/elasticsearch"
es_server = Popen([path_to_elastic],
                    stdout=PIPE, stderr=STDOUT,
                    preexec_fn=os.setuid(1)  # as daemon
                    )

config = {'host':'localhost', 'port':9200}
time.sleep(10)
es = Elasticsearch([config])
print('elastic serach ping :', es.ping())

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
elastic serach ping : True


In [7]:
INDEX_CONFIG = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "decompound_mode": "mixed",
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", 
            "properties": {
                "id": {"type": "int"},
                "title": {"type": "text", "analyzer":"nori_analyzer"},
                "context": {"type": "text", "analyzer":"nori_analyzer"},
                }
            }
        }

In [8]:
INDEX_NAME = "wiki"
if not es.indices.exists(index=INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body=INDEX_CONFIG, ignore=400)

<ipython-input-8-30f97ff7a951>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_CONFIG, ignore=400)


In [9]:
from tqdm import tqdm
for i, rec in enumerate(tqdm(wiki_contexts)):
    try:
        index_status = es.index(index=INDEX_NAME, id=i, body=rec)
    except:
        print(f'Unable to load document {i}.')
            
n_records = es.count(index=INDEX_NAME)['count']
print(f'Succesfully loaded {n_records} into {INDEX_NAME}')

  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-9-6b84f5f80725>:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  index_status = es.index(index=INDEX_NAME, id=i, body=rec)
100%|██████████| 60613/60613 [03:53<00:00, 259.71it/s]


Succesfully loaded 60613 into wiki


In [10]:
from datasets import load_from_disk
train_path = "/opt/ml/mrc-level2-nlp-11/data/train_dataset"
data = load_from_disk(train_path)
train_data = data["train"]
print(train_data)

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})


In [15]:
match_cnt = 0
for datum in tqdm(train_data):
    question_text = datum["question"]
    context = datum["context"]
    query = {
            'query': {
                'match': {
                    'context': question_text
                    }
                }
            }
    
    result = es.search(index=INDEX_NAME, body=query, size=5)
    print(result["hits"]["hits"])
    break
    for res in result["hits"]["hits"]:
        if res["_source"]["context"] == context:
            match_cnt += 1
            break
print(f"matching score is {match_cnt/3952:.3f}")

  0%|          | 0/3952 [00:00<?, ?it/s]<ipython-input-15-a8597dc19e1c>:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index=INDEX_NAME, body=query, size=5)
  0%|          | 0/3952 [00:00<?, ?it/s]

[{'_index': 'wiki', '_type': '_doc', '_id': '2283', '_score': 19.043957, '_source': {'id': 2283, 'title': '정부 형태', 'context': '이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권한이 2인에게로 분리되어 있지 않으므로 이원집정부제는 아니다.\n\n \n* 나우루\n* 남아프리카 공화국\n* 마셜 제도\n* 마카오\n* 미얀마\n* 미크로네시아 연방\n* 보츠와나\n* 산마리노\n* 수리남\n* 키리바시\n* 홍콩'}}, {'_index': 'wiki', '_type': '_doc', '_id': '6837', '_score': 19.043957, '_source': {'id': 6837, 'title': '정부 형태', 'context': '이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권

In [12]:
'''
top-1 : 0.334
top-5 : 0.562
top-10 : 0.613
top-20 : 0.656
top-30 : 0.680
top-50 : 0.709
top-100: 0.743
'''

'\ntop-1 : 0.334\ntop-5 : 0.562\ntop-10 : 0.613\ntop-20 : 0.656\ntop-30 : 0.680\ntop-50 : 0.709\ntop-100: 0.743\n'

In [13]:
###matrix -> LSA 
     doc1 doc2 doc3
term1
temr2
...
nxm
LSA,
SVD -> n x k, k x k, k xm
kxm -> m : 문서, k : latent feature(저자, 주제, )
문서마다 k 벡터로 나타나진다 -> k : 3

clustering : 10
query 


IndentationError: unexpected indent (<ipython-input-13-140961a91853>, line 2)